# Chapter 24: Class Decorators and Subclass Hooks

Class decorators and `__init_subclass__` are powerful tools for modifying classes after
they are created. They sit in the sweet spot between simple function decorators and the
full complexity of metaclasses, handling the vast majority of class-customization needs
with clear, readable code.

## Topics Covered
- **Class decorators**: Modifying classes after creation
- **Adding methods and attributes** with decorators
- **`@dataclass`** as the canonical class decorator example
- **`__init_subclass__`**: Hook called when a class is subclassed
- **Plugin registration** with `__init_subclass__`
- **Comparing**: Class decorator vs metaclass vs `__init_subclass__`
- **Decorator stacking** order and composition
- **Practical**: Auto-registering command handlers

## Class Decorators: Modifying Classes After Creation

A class decorator is a callable that takes a class object as its argument and returns
a (usually modified) class. The syntax `@decorator` before `class Foo:` is equivalent
to `Foo = decorator(Foo)` after the class body executes.

In [ ]:
from typing import Any, TypeVar

T = TypeVar("T")


def add_repr(cls: type[T]) -> type[T]:
    """Class decorator that adds a __repr__ based on __init__ parameters."""

    def __repr__(self: Any) -> str:
        attrs = ", ".join(
            f"{k}={v!r}"
            for k, v in self.__dict__.items()
            if not k.startswith("_")
        )
        return f"{type(self).__name__}({attrs})"

    cls.__repr__ = __repr__  # type: ignore[attr-defined]
    return cls


def add_eq(cls: type[T]) -> type[T]:
    """Class decorator that adds __eq__ based on public attributes."""

    def __eq__(self: Any, other: Any) -> bool:
        if type(self) is not type(other):
            return NotImplemented
        return self.__dict__ == other.__dict__

    cls.__eq__ = __eq__  # type: ignore[attr-defined]
    return cls


@add_repr
@add_eq
class Color:
    def __init__(self, r: int, g: int, b: int) -> None:
        self.r = r
        self.g = g
        self.b = b


c1 = Color(255, 128, 0)
c2 = Color(255, 128, 0)
c3 = Color(0, 0, 0)

print(f"c1 = {c1!r}")
print(f"c2 = {c2!r}")
print(f"c1 == c2: {c1 == c2}")
print(f"c1 == c3: {c1 == c3}")

## Adding Methods and Attributes with Decorators

Class decorators can inject methods, class attributes, or even modify the class
hierarchy. A parameterized class decorator (a decorator factory) takes arguments
and returns the actual decorator.

In [ ]:
from typing import Any, TypeVar
from datetime import datetime, timezone

T = TypeVar("T")


def timestamped(cls: type[T]) -> type[T]:
    """Inject created_at tracking into a class."""
    original_init = cls.__init__

    def new_init(self: Any, *args: Any, **kwargs: Any) -> None:
        original_init(self, *args, **kwargs)
        self.created_at = datetime.now(timezone.utc)

    cls.__init__ = new_init  # type: ignore[attr-defined]
    return cls


def with_version(version: str):
    """Parameterized class decorator that adds a version attribute."""
    def decorator(cls: type[T]) -> type[T]:
        cls.version = version  # type: ignore[attr-defined]

        def get_info(self: Any) -> str:
            return f"{type(self).__name__} v{cls.version}"  # type: ignore[attr-defined]

        cls.get_info = get_info  # type: ignore[attr-defined]
        return cls
    return decorator


@timestamped
@with_version("2.1.0")
class UserService:
    def __init__(self, name: str) -> None:
        self.name = name


svc = UserService("auth")
print(f"svc.name = {svc.name!r}")
print(f"svc.created_at = {svc.created_at}")
print(f"UserService.version = {UserService.version!r}")
print(f"svc.get_info() = {svc.get_info()!r}")

## @dataclass as the Canonical Class Decorator

`@dataclasses.dataclass` is the most widely used class decorator in the standard library.
It inspects class annotations to automatically generate `__init__`, `__repr__`, `__eq__`,
and other methods. Understanding how it works demystifies class decorators in general.

In [ ]:
from dataclasses import dataclass, field, fields


@dataclass(frozen=True, order=True)
class Coordinate:
    """An immutable, orderable 2D coordinate."""
    x: float
    y: float
    label: str = field(default="", compare=False, repr=False)


p1 = Coordinate(1.0, 2.0, "origin-ish")
p2 = Coordinate(3.0, 4.0)
p3 = Coordinate(1.0, 2.0)

print(f"p1 = {p1!r}")
print(f"p2 = {p2!r}")
print(f"p1 == p3: {p1 == p3}")  # True (label excluded from compare)
print(f"p1 < p2:  {p1 < p2}")   # True (compared as tuples of (x, y))

# Inspect what @dataclass generated
print(f"\nFields: {[f.name for f in fields(Coordinate)]}")
print(f"Generated methods: {[m for m in dir(Coordinate) if not m.startswith('_') or m in ('__init__', '__repr__', '__eq__', '__lt__')]}")

# Frozen means immutable
try:
    p1.x = 99
except AttributeError as e:
    print(f"\nCannot mutate frozen: {e}")

## `__init_subclass__`: Hook Called When a Class Is Subclassed

Introduced in Python 3.6, `__init_subclass__` is a class method that is automatically
called on the **parent** class whenever it is subclassed. This provides a simple hook
for customizing subclass creation without needing a metaclass.

Key points:
- Defined on the parent class, called when a child is created
- Receives the new subclass as its first argument (after `cls`)
- Can accept keyword arguments from the class statement

In [ ]:
class Validated:
    """Base class that enforces subclasses must define required_fields."""

    def __init_subclass__(cls, *, required_fields: tuple[str, ...] = (), **kwargs: object) -> None:
        super().__init_subclass__(**kwargs)
        cls._required_fields = required_fields
        print(f"  Registered {cls.__name__} with required fields: {required_fields}")

        # Inject a validate() method
        def validate(self: object) -> bool:
            missing = [
                f for f in cls._required_fields
                if not hasattr(self, f) or getattr(self, f) is None
            ]
            if missing:
                raise ValueError(f"Missing required fields: {missing}")
            return True

        cls.validate = validate  # type: ignore[attr-defined]


print("Defining subclasses:")


class User(Validated, required_fields=("name", "email")):
    def __init__(self, name: str | None = None, email: str | None = None) -> None:
        self.name = name
        self.email = email


class Product(Validated, required_fields=("sku", "price")):
    def __init__(self, sku: str | None = None, price: float | None = None) -> None:
        self.sku = sku
        self.price = price


# Valid instance
user = User("Alice", "alice@example.com")
print(f"\nuser.validate() = {user.validate()}")

# Invalid instance -- missing required field
incomplete = User("Bob", None)
try:
    incomplete.validate()
except ValueError as e:
    print(f"Validation error: {e}")

## Plugin Registration with `__init_subclass__`

One of the most practical uses of `__init_subclass__` is automatic plugin registration.
Subclasses register themselves simply by existing -- no manual registration step needed.

In [ ]:
from typing import ClassVar


class Serializer:
    """Base serializer that auto-registers subclasses by format name."""
    _registry: ClassVar[dict[str, type["Serializer"]]] = {}

    def __init_subclass__(cls, *, format_name: str = "", **kwargs: object) -> None:
        super().__init_subclass__(**kwargs)
        if format_name:
            cls._registry[format_name] = cls
            cls.format_name = format_name  # type: ignore[attr-defined]

    @classmethod
    def get_serializer(cls, format_name: str) -> "Serializer":
        """Look up a serializer by format name."""
        if format_name not in cls._registry:
            raise ValueError(f"Unknown format: {format_name!r}")
        return cls._registry[format_name]()

    def serialize(self, data: dict) -> str:
        raise NotImplementedError


class JSONSerializer(Serializer, format_name="json"):
    def serialize(self, data: dict) -> str:
        import json
        return json.dumps(data)


class CSVSerializer(Serializer, format_name="csv"):
    def serialize(self, data: dict) -> str:
        header = ",".join(data.keys())
        values = ",".join(str(v) for v in data.values())
        return f"{header}\n{values}"


class XMLSerializer(Serializer, format_name="xml"):
    def serialize(self, data: dict) -> str:
        items = "".join(f"<{k}>{v}</{k}>" for k, v in data.items())
        return f"<record>{items}</record>"


# All serializers registered automatically
print(f"Registered formats: {list(Serializer._registry.keys())}")

# Use the registry to dispatch
sample_data = {"name": "Alice", "age": 30, "city": "Paris"}

for fmt in ["json", "csv", "xml"]:
    s = Serializer.get_serializer(fmt)
    print(f"\n[{fmt}]:\n{s.serialize(sample_data)}")

## Comparing: Class Decorator vs Metaclass vs `__init_subclass__`

Python offers three main mechanisms for customizing class creation. Here is when
to use each one:

| Mechanism | Applied To | When It Runs | Complexity |
|-----------|-----------|-------------|------------|
| Class decorator | The decorated class only | After class body executes | Low |
| `__init_subclass__` | All subclasses of a base | When each subclass is created | Medium |
| Metaclass | All instances of the metaclass | During class creation | High |

**Rule of thumb**: Start with class decorators. Move to `__init_subclass__` when you
need automatic behavior in all subclasses. Only reach for metaclasses when you need
to control the class creation process itself (e.g., modifying `__new__` or the class
namespace).

In [ ]:
from typing import Any


# Approach 1: Class decorator -- explicit, opt-in
def add_greeting(cls: type) -> type:
    cls.greet = lambda self: f"Hello from {type(self).__name__}!"  # type: ignore[attr-defined]
    return cls


@add_greeting
class ServiceA:
    pass


# Approach 2: __init_subclass__ -- automatic for all subclasses
class GreetingBase:
    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)
        cls.greet = lambda self: f"Hello from {type(self).__name__}!"  # type: ignore[attr-defined]


class ServiceB(GreetingBase):
    pass


class ServiceC(GreetingBase):
    pass


# Approach 3: Metaclass -- most powerful, most complex
class GreetingMeta(type):
    def __new__(mcs, name: str, bases: tuple, namespace: dict[str, Any]) -> type:
        cls = super().__new__(mcs, name, bases, namespace)
        cls.greet = lambda self: f"Hello from {type(self).__name__}!"  # type: ignore[attr-defined]
        return cls


class ServiceD(metaclass=GreetingMeta):
    pass


# All three approaches achieve the same result
for cls in [ServiceA, ServiceB, ServiceC, ServiceD]:
    instance = cls()
    print(f"{cls.__name__}: {instance.greet()}")

print(f"\nServiceB uses: __init_subclass__ (auto for all subclasses)")
print(f"ServiceA uses: class decorator (explicit opt-in)")
print(f"ServiceD uses: metaclass (full control, but rarely needed)")

## Decorator Stacking Order and Composition

When multiple class decorators are stacked, they are applied **bottom-up** (innermost
first). This is the same rule as function decorators. Understanding the order matters
when decorators depend on or modify each other's work.

In [ ]:
from typing import TypeVar

T = TypeVar("T")


def decorator_a(cls: type[T]) -> type[T]:
    """Adds method_a and records application order."""
    order = getattr(cls, "_decorator_order", [])
    cls._decorator_order = order + ["A"]  # type: ignore[attr-defined]
    cls.method_a = lambda self: "from A"  # type: ignore[attr-defined]
    print(f"  Applied decorator A to {cls.__name__}")
    return cls


def decorator_b(cls: type[T]) -> type[T]:
    """Adds method_b and records application order."""
    order = getattr(cls, "_decorator_order", [])
    cls._decorator_order = order + ["B"]  # type: ignore[attr-defined]
    cls.method_b = lambda self: "from B"  # type: ignore[attr-defined]
    print(f"  Applied decorator B to {cls.__name__}")
    return cls


def decorator_c(cls: type[T]) -> type[T]:
    """Adds method_c and records application order."""
    order = getattr(cls, "_decorator_order", [])
    cls._decorator_order = order + ["C"]  # type: ignore[attr-defined]
    cls.method_c = lambda self: "from C"  # type: ignore[attr-defined]
    print(f"  Applied decorator C to {cls.__name__}")
    return cls


# Stacking: bottom-up application order
# Equivalent to: MyClass = decorator_a(decorator_b(decorator_c(MyClass)))
print("Applying decorators (bottom-up):")


@decorator_a  # Applied third (outermost)
@decorator_b  # Applied second
@decorator_c  # Applied first (innermost)
class MyClass:
    pass


obj = MyClass()
print(f"\nApplication order: {MyClass._decorator_order}")
print(f"obj.method_a() = {obj.method_a()!r}")
print(f"obj.method_b() = {obj.method_b()!r}")
print(f"obj.method_c() = {obj.method_c()!r}")

## Practical: Auto-Registering Command Handlers

This practical example builds a command dispatcher that uses `__init_subclass__`
to automatically register handler classes, and a class decorator to tag individual
methods as command handlers. This is a pattern found in CLI frameworks, chatbots,
and game engines.

In [ ]:
from typing import Any, Callable, ClassVar
import functools


# Step 1: A method decorator to tag individual methods as commands
def command(name: str | None = None, description: str = ""):
    """Mark a method as a command handler."""
    def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
        func._command_name = name or func.__name__  # type: ignore[attr-defined]
        func._command_desc = description  # type: ignore[attr-defined]
        return func
    return decorator


# Step 2: Base class with __init_subclass__ for auto-registration
class CommandHandler:
    """Base class that auto-discovers @command methods in subclasses."""
    _handlers: ClassVar[dict[str, tuple[type, Callable[..., Any], str]]] = {}

    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)
        # Scan the subclass for methods decorated with @command
        for attr_name in dir(cls):
            method = getattr(cls, attr_name, None)
            if callable(method) and hasattr(method, "_command_name"):
                cmd_name = method._command_name
                cmd_desc = method._command_desc
                cls._handlers[cmd_name] = (cls, method, cmd_desc)

    @classmethod
    def dispatch(cls, command_str: str) -> str:
        """Parse and dispatch a command string."""
        parts = command_str.strip().split()
        cmd_name = parts[0]
        args = parts[1:]

        if cmd_name not in cls._handlers:
            return f"Unknown command: {cmd_name!r}"

        handler_cls, method, _ = cls._handlers[cmd_name]
        instance = handler_cls()
        return method(instance, *args)

    @classmethod
    def help_text(cls) -> str:
        """Generate help text from all registered commands."""
        lines = ["Available commands:"]
        for name, (handler_cls, _, desc) in sorted(cls._handlers.items()):
            lines.append(f"  {name:15s} - {desc} [{handler_cls.__name__}]")
        return "\n".join(lines)


print("Defining command handler classes...")

In [ ]:
# Step 3: Define handler subclasses -- they register automatically

class FileCommands(CommandHandler):
    """File-related commands."""

    @command(name="ls", description="List files in a directory")
    def list_files(self, path: str = ".") -> str:
        return f"Listing files in {path!r}: file1.txt, file2.py, data.csv"

    @command(name="cat", description="Display file contents")
    def show_file(self, filename: str) -> str:
        return f"Contents of {filename!r}: [simulated file content]"


class SystemCommands(CommandHandler):
    """System-related commands."""

    @command(name="status", description="Show system status")
    def system_status(self) -> str:
        return "System OK | CPU: 45% | Memory: 62%"

    @command(name="uptime", description="Show system uptime")
    def system_uptime(self) -> str:
        return "Uptime: 14 days, 3 hours, 22 minutes"


class UserCommands(CommandHandler):
    """User management commands."""

    @command(name="whoami", description="Show current user")
    def who_am_i(self) -> str:
        return "Current user: admin (role: superuser)"


# Show the auto-generated help text
print(CommandHandler.help_text())

# Dispatch some commands
print()
for cmd in ["ls /home", "cat readme.md", "status", "whoami", "unknown_cmd"]:
    result = CommandHandler.dispatch(cmd)
    print(f"  > {cmd}")
    print(f"    {result}")

## Summary

### Key Takeaways

| Mechanism | Best For | Example |
|-----------|----------|--------|
| **Class decorator** | One-off modifications, adding methods/attrs | `@add_repr`, `@dataclass` |
| **Parameterized decorator** | Configurable class modifications | `@with_version("2.0")` |
| **`__init_subclass__`** | Auto-registration, subclass validation | Plugin systems, serializers |
| **Metaclass** | Deep class creation control | ORMs, API frameworks |
| **`@command` + `__init_subclass__`** | Command/plugin dispatch | CLI tools, chatbots |

### Best Practices
- Start with class decorators for simple, explicit modifications
- Use `__init_subclass__` when all subclasses should automatically participate
- Remember decorator stacking order: bottom-up (innermost applied first)
- Reserve metaclasses for cases where you truly need to intercept class creation
- Combine method decorators with `__init_subclass__` for powerful plugin patterns
- Always call `super().__init_subclass__(**kwargs)` to support cooperative inheritance